<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M4_LSTM_multiclass_text_classification_PyTorch_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Text Classification Hate Speech Detection

<img src="https://blog.colinbreck.com/content/images/size/w2000/2021/01/twitter-a-love-hate-relationship-1.png" width="500">


# Importing Libraries

In [ ]:
# !pip install spacy
!python -m spacy download en_core_web_md -q

2023-02-14 04:52:13.320925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 04:52:14.253769: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 04:52:14.253888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-14 04:52:14.253909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
import pandas as pd
import numpy as np

import spacy
import re

#First download SpaCy's en_core_web_md model then load 
#!python -m spacy download en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load() 

import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English


import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier,XGBRFClassifier

%matplotlib inline
import warnings
warnings.filterwarnings(action = 'ignore')

Data Description:
==============

* count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were     determined to be unreliable by CF).

* hate_speech = number of CF users who judged the tweet to be hate speech.

* offensive_language = number of CF users who judged the tweet to be offensive.

* neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

* class = class label for majority of CF users.

   0 - hate speech
   1 - offensive language 
   2 - neither

* tweet = text data(tweet)

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/aaubs/ds-master/main/data/HateSpeechData_pre.csv')
df.head()

,count,hate_speech,offensive_language,neither,class,tweet,text length,final_tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,140,woman shouldn t complain clean house amp man t...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,85,boy dats cold tyga dwn bad cuffin dat hoe st p...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,120,dawg fuck bitch start cry confuse shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,62,look like tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,137,shit hear true faker bitch tell ya


# Preprocessing of the tweets

In [ ]:
# Create our list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
#retweet, fav, follow friday

nlp.Defaults.stop_words |= {"#ff", "ff", "rt",}

In [ ]:
def preprocess(tweet):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(regex_pat, ' ')

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(regex_pat, '')

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(giant_url_regex, '')
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ") 
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ')                   
    # remove leading and trailing whitespace 
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with number  
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
 
    # tokenizing
    tokenized_tweet = newtweet.apply(lambda x: x.split()) 
    
 
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    
    return tweets_p
   
processed_tweets = preprocess(df['tweet'])

<ipython-input-58-ff804d7f9f6b>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  punc_remove = tweets.str.replace("[^a-zA-Z]", " ")
<ipython-input-58-ff804d7f9f6b>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  newtweet=punc_remove.str.replace(r'\s+', ' ')
<ipython-input-58-ff804d7f9f6b>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
<ipython-input-58-ff804d7f9f6b>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')


In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(processed_tweets):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(processed_tweets)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words ]

    # return preprocessed tokens
    return ' '.join(mytokens)

In [ ]:
%%time
df['final_tweet'] = processed_tweets.apply(lambda x: spacy_tokenizer(x))

CPU times: user 2min 36s, sys: 485 ms, total: 2min 36s
Wall time: 2min 39s


In [ ]:
df.head()

,count,hate_speech,offensive_language,neither,class,tweet,text length,final_tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,140,woman shouldn t complain clean house amp man t...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,85,boy dat cold tyga dwn bad cuffin dat hoe st place
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,120,dawg fuck bitch start cry confuse shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,62,look like tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,137,shit hear true faker bitch tell ya


In [ ]:
#keeping only relevant columns and calculating sentence lengths
df['tweet_length'] = df['tweet'].apply(lambda x: len(x.split()))
df.head()

,count,hate_speech,offensive_language,neither,class,tweet,text length,final_tweet,tweet_length
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,140,woman shouldn t complain clean house amp man t...,25
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,85,boy dat cold tyga dwn bad cuffin dat hoe st place,16
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,120,dawg fuck bitch start cry confuse shit,21
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,62,look like tranny,9
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,137,shit hear true faker bitch tell ya,26


In [ ]:
#mean sentence length
np.mean(df['tweet_length'])

14.117015696243392

In [ ]:
#mean sentence length
np.max(df['tweet_length'])

52

In [ ]:
#tokenization
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [ ]:
df.dropna(subset=['final_tweet'], inplace=True)

In [ ]:
df.shape

(24783, 9)

In [ ]:
#count number of occurences of each word
counts = Counter()
for index, row in df.iterrows():
    counts.update(tokenize(row['final_tweet']))

In [ ]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 17054
num_words after: 7681


In [ ]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
len(words)

7683

In [ ]:
def encode_sentence(text, vocab2index, N=52):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
df['encoded'] = df['final_tweet'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df.head()

<ipython-input-73-4552c063a01e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df['encoded'] = df['final_tweet'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))


,count,hate_speech,offensive_language,neither,class,tweet,text length,final_tweet,tweet_length,encoded
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,140,woman shouldn t complain clean house amp man t...,25,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0..."
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,85,boy dat cold tyga dwn bad cuffin dat hoe st place,16,"[[11, 12, 13, 14, 15, 16, 17, 12, 18, 19, 20, ..."
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,120,dawg fuck bitch start cry confuse shit,21,"[[21, 22, 23, 24, 25, 26, 27, 0, 0, 0, 0, 0, 0..."
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,62,look like tranny,9,"[[28, 29, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,137,shit hear true faker bitch tell ya,26,"[[27, 31, 32, 33, 23, 34, 35, 0, 0, 0, 0, 0, 0..."


In [ ]:
#check how balanced the dataset is
Counter(df['class'])

Counter({2: 4163, 1: 19190, 0: 1430})

In [ ]:
df['encoded'][0][0]

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0])

In [ ]:
X = list(df['encoded'])
y = list(df['class'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [ ]:
len(X_train[2][0])

52

In [ ]:
X_train[3][1]

8

#### Pytorch Dataset

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [ ]:
train_ds

In [ ]:
def train_model(model, epochs=10, lr=0.01):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 1 == 0:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
batch_size = 10
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
df.head()

,count,hate_speech,offensive_language,neither,class,tweet,text length,final_tweet,tweet_length,encoded
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,140,woman shouldn t complain clean house amp man t...,25,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0..."
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,85,boy dats cold tyga dwn bad cuffin dat hoe st p...,16,"[[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, ..."
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,120,dawg fuck bitch start cry confuse shit,21,"[[22, 23, 24, 25, 26, 27, 28, 0, 0, 0, 0, 0, 0..."
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,62,look like tranny,9,"[[29, 30, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,137,shit hear true faker bitch tell ya,26,"[[28, 32, 33, 34, 24, 35, 36, 0, 0, 0, 0, 0, 0..."


### LSTM with fixed length input

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [ ]:
train_model(model_fixed, epochs=10, lr=0.1)

train loss 0.772, val loss 0.729, val accuracy 0.702, and val rmse 0.562
train loss 0.729, val loss 0.666, val accuracy 0.740, and val rmse 0.509
train loss 0.703, val loss 0.721, val accuracy 0.763, and val rmse 0.458
train loss 0.765, val loss 1.008, val accuracy 0.753, and val rmse 0.473
train loss 0.865, val loss 0.765, val accuracy 0.733, and val rmse 0.499
train loss 0.833, val loss 0.738, val accuracy 0.759, and val rmse 0.463
train loss 0.810, val loss 0.693, val accuracy 0.761, and val rmse 0.460
train loss 0.825, val loss 1.064, val accuracy 0.763, and val rmse 0.457
train loss 0.800, val loss 0.774, val accuracy 0.763, and val rmse 0.458
train loss 0.797, val loss 0.787, val accuracy 0.763, and val rmse 0.457
